In [4]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

print("=" * 60)
print("REAL-WORLD SQL APPLICATIONS")
print("=" * 60)
print("E-Commerce Database with Analytics")

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create customers table
cursor.execute("""
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    city TEXT,
    signup_date TEXT
)
""")

# Create products table
cursor.execute("""
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    name TEXT,
    category TEXT,
    price REAL
)
""")

# Create orders table
cursor.execute("""
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    total_amount REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)
""")

# Create order_items table
cursor.execute("""
CREATE TABLE order_items (
    item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    price REAL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
""")

# Insert customers
customers = [
    (1, 'Alice Johnson', 'alice@example.com', 'New York', '2023-01-15'),
    (2, 'Bob Smith', 'bob@example.com', 'Los Angeles', '2023-02-20'),
    (3, 'Carol Davis', 'carol@example.com', 'Chicago', '2023-03-10'),
    (4, 'David Wilson', 'david@example.com', 'Houston', '2023-04-05'),
    (5, 'Eve Taylor', 'eve@example.com', 'Phoenix', '2023-05-12'),
]

# Insert products
products = [
    (1, 'Laptop Pro', 'Electronics', 1500),
    (2, 'USB Mouse', 'Electronics', 25),
    (3, 'Office Desk', 'Furniture', 350),
    (4, 'Ergonomic Chair', 'Furniture', 200),
    (5, 'Monitor 27"', 'Electronics', 400),
    (6, 'Keyboard Mechanical', 'Electronics', 120),
]

# Insert orders
orders = [
    (1, 1, '2023-06-01', 1525),
    (2, 2, '2023-06-05', 425),
    (3, 1, '2023-07-10', 1900),
    (4, 3, '2023-07-15', 550),
    (5, 2, '2023-08-20', 145),
    (6, 4, '2023-09-01', 2050),
    (7, 1, '2023-09-15', 2400),
]

# Insert order items
order_items = [
    (1, 1, 1, 1, 1500),
    (2, 1, 2, 1, 25),
    (3, 2, 5, 1, 400),
    (4, 2, 6, 1, 25),
    (5, 3, 1, 1, 1500),
    (6, 3, 3, 1, 400),
    (7, 4, 3, 1, 350),
    (8, 4, 4, 1, 200),
    (9, 5, 2, 5, 25),
    (10, 6, 1, 1, 1500),
    (11, 6, 5, 1, 400),
    (12, 6, 6, 2, 120),
    (13, 7, 1, 1, 1500),
    (14, 7, 4, 3, 200),
    (15, 7, 2, 4, 25),
]

cursor.executemany("INSERT INTO customers VALUES (?, ?, ?, ?, ?)", customers)
cursor.executemany("INSERT INTO products VALUES (?, ?, ?, ?)", products)
cursor.executemany("INSERT INTO orders VALUES (?, ?, ?, ?)", orders)
cursor.executemany("INSERT INTO order_items VALUES (?, ?, ?, ?, ?)", order_items)
conn.commit()

print("✓ Database setup complete")
print(f"  - {len(customers)} customers")
print(f"  - {len(products)} products")
print(f"  - {len(orders)} orders")
print(f"  - {len(order_items)} order items")

REAL-WORLD SQL APPLICATIONS
E-Commerce Database with Analytics
✓ Database setup complete
  - 5 customers
  - 6 products
  - 7 orders
  - 15 order items


## 1. Customer Lifetime Value (CLV)

CLV measures the total revenue generated from a customer over their relationship with the company.

**Metrics:**
- Total orders: How many times they purchased
- Total spent: Sum of order amounts
- Average order value: Total spent / number of orders

In [8]:
# Calculate Customer Lifetime Value (CLV)
print("\n" + "=" * 60)
print("CUSTOMER LIFETIME VALUE ANALYSIS")
print("=" * 60)

query = """
SELECT 
    c.name as customer_name,
    COUNT(o.order_id) as total_orders,
    SUM(o.total_amount) as total_spent,
    ROUND(SUM(o.total_amount) / COUNT(o.order_id), 2) as avg_order_value
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id , c.name
ORDER BY total_spent DESC
"""

df_clv = pd.read_sql_query(query, conn)
print("\nCustomer Lifetime Value Summary:")
print(df_clv.to_string(index=False))

print("\n" + "-" * 60)
print("Key Insights:")
print(f"  - Total customers analyzed: {len(df_clv)}")
print(f"  - Highest CLV: {df_clv.iloc[0]['customer_name']} (${df_clv.iloc[0]['total_spent']:.2f})")
print(f"  - Average customer value: ${df_clv['total_spent'].mean():.2f}")


CUSTOMER LIFETIME VALUE ANALYSIS

Customer Lifetime Value Summary:
customer_name  total_orders  total_spent  avg_order_value
Alice Johnson             3       5825.0          1941.67
 David Wilson             1       2050.0          2050.00
    Bob Smith             2        570.0           285.00
  Carol Davis             1        550.0           550.00
   Eve Taylor             0          NaN              NaN

------------------------------------------------------------
Key Insights:
  - Total customers analyzed: 5
  - Highest CLV: Alice Johnson ($5825.00)
  - Average customer value: $2248.75
